In [ ]:
#default_exp functional

# Functional

> Test function output values against Wolfram | Alpha

In [ ]:
#export
import wolframalpha
from fastcore.test import *
from torch import nn
import torch
import torch.nn.functional as F
import re
import numpy as np
import pickle

In [ ]:
#export
class WolframTester():
    
    def __init__(self, api_key, libdl):
        self.key = api_key
        self.libdl = libdl
        self.cache = None
        
    
    def query(self, expr):
        client = wolframalpha.Client(self.key)
        res = client.query(expr)
        vals = list()
        for pod in res.pods:
            if pod['@title'] == 'Result':
                val = float(pod['subpod']['plaintext'][:6])
                vals.append(val)
        return np.array(val)
    
    
    def test(self, fn, fn_expr, xs, shape):
        
        if (self.cache is not None):
            self.test_cache(fn, fn_expr, xs, shape)
            return
    
        if (self.libdl == 'torch'):
            ys = fn(xs).cpu().numpy()
            test_eq(ys.shape, shape)
            _xs = xs.cpu().detach().numpy().flatten()
        
        reals = list()

        for x in _xs:
            expr = re.sub('x', str(x), fn_expr)
            res = self.query(expr)
            reals.append(res)
        
        reals = np.array(reals).reshape(shape)
        np.testing.assert_allclose(ys, reals, rtol=1e-2, atol=1e-5)
        
        self.cache = {fn_expr : (xs, reals)}
        self.save_cache(fn.__name__)
        
    
    def test_cache(self, fn, fn_expr, xs, shape):
        self.load_cache(fn.__name__)
        xs, reals = self.cache[fn_expr]
        
        if (self.libdl == 'torch'):
            ys = fn(xs).cpu().numpy()
            test_eq(ys.shape, shape)
            xs = xs.cpu().detach().numpy().flatten()
        
        np.testing.assert_allclose(ys, reals, rtol=1e-2, atol=1e-5)
        
        
    def save_cache(self, name):
        with open(f'{name}.pkl', 'wb') as f:
            pickle.dump(self.cache, f, pickle.HIGHEST_PROTOCOL)
            print("Stored Cache")

    def load_cache(self, name):
        with open(f'{name}.pkl', 'rb') as f:
            self.cache = pickle.load(f)
            print("Loaded Cache")

In [ ]:
function = torch.tanh
xs = torch.tensor([[[[-10, -8, -6, -4, -2], [0, 2, 4, 6, 8]]]], dtype=torch.float32)
shape = (1, 1, 2, 5)

In [ ]:
tester = WolframTester('YOUR_API_KEY', 'torch')
tester = WolframTester('QYU645-4EGHX3JVLE', 'torch')

The first time the test function is executed, it queries the Wolfram API. This function call will be slow. 

In [ ]:
%%time
tester.test(function, 'tanh(x)', xs, shape)

Stored Cache
CPU times: user 226 ms, sys: 33.9 ms, total: 260 ms
Wall time: 41.5 s


But now, the inputs and expected outputs are cached on disk in `.pkl` files.

In [ ]:
! ls *.pkl

tanh.pkl


So subsequent function calls are much faster!

In [ ]:
%%time
tester.test(function, 'tanh(x)', xs, shape)

Loaded Cache
CPU times: user 2.63 ms, sys: 2.77 ms, total: 5.4 ms
Wall time: 9.3 ms
